In [ ]:
from os import makedirs
from os.path import join, splitext, basename
DIR_ROOT = "../data/test_dataset"
DIR_RENDITIONS = join(DIR_ROOT, "output/renditions")
DIR_CSVS = join(DIR_ROOT, "output/csvs")
DIR_VISUALIZATIONS = join(DIR_ROOT, "output/visualizations/region_props")
DIR_2ND_ORDER = join(DIR_CSVS, "second_order_entropy")

makedirs(DIR_RENDITIONS, exist_ok=True)
makedirs(DIR_CSVS, exist_ok=True)
makedirs(DIR_VISUALIZATIONS, exist_ok=True)
makedirs(DIR_2ND_ORDER, exist_ok=True)

# generate 8k random rotations

In [ ]:
from numpy.random import MT19937
import numpy as np
from scipy.spatial.transform import Rotation

RANDOM_SEED = 42
SAMPLE_SIZE = 8000
EULER_ORDER = "YZX"


rng = np.random.Generator(MT19937(RANDOM_SEED))
euler_array = Rotation.random(SAMPLE_SIZE, random_state=rng).as_euler(seq=EULER_ORDER, degrees=True)

euler_array

8788

# Populate the Euler Angles dataframe

In [ ]:
import polars as pl
from scipy.spatial.transform import Rotation
from glob import glob


DIR_OBJECTS = "../data/PrefViews Null 3D objects"
CSV_ROTATION = join(DIR_CSVS, "rotation_data.csv")

toy_paths = glob(join(DIR_OBJECTS, "*.obj"))
toys = [splitext(basename(file))[0] for file in toy_paths]

# Create DataFrames and join
toys_series = [toys[i%len(toys)] for i in range(SAMPLE_SIZE)]
df_toys = pl.DataFrame({"Object": toys_series})
df_angles = pl.DataFrame({
    "Euler_X": euler_array[:, 2],
    "Euler_Y": euler_array[:, 0],
    "Euler_Z": euler_array[:, 1],
})

# Cross join = every toy with every angle
DF_ROTATIONS = pl.DataFrame({
    'Object': toys_series,
    "Euler_X": euler_array[:, 2],
    "Euler_Y": euler_array[:, 0],
    "Euler_Z": euler_array[:, 1],
})

#Add index

DF_ROTATIONS = DF_ROTATIONS.with_columns(
    pl.arange(0, pl.len()).alias("Index")
)
# Add primary key
DF_ROTATIONS = DF_ROTATIONS.with_columns(
    pl.concat_str([
        pl.col("Index"),
        pl.lit("_"),
        pl.col("Object")
    ]).alias("KID_IDFrameKey")
).select([
    "Index",
    "KID_IDFrameKey",
    pl.all().exclude(["Index", "KID_IDFrameKey"])
])

DF_ROTATIONS.write_csv(CSV_ROTATION)
DF_ROTATIONS

Index,KID_IDFrameKey,Object,Euler_X,Euler_Y,Euler_Z
i64,str,str,i64,i64,i64
0,"""0_CribNovel""","""CribNovel""",0,0,0
1,"""1_CribNovel""","""CribNovel""",-30,0,0
2,"""2_CribNovel""","""CribNovel""",-60,0,0
3,"""3_CribNovel""","""CribNovel""",-90,0,0
4,"""4_CribNovel""","""CribNovel""",-120,0,0
…,…,…,…,…,…
8783,"""8783_CameraNovel""","""CameraNovel""",180,30,30
8784,"""8784_CameraNovel""","""CameraNovel""",120,30,30
8785,"""8785_CameraNovel""","""CameraNovel""",90,30,30


# Generate the renditions for each rotation

In [ ]:
from novel_toy_tools.utils.generate_renditions import generate_renditions_rot_mat

generate_renditions_rot_mat(CSV_ROTATION, DIR_RENDITIONS, DIR_OBJECTS)

Saving renditions to ../data/test_dataset/output/renditions


Generating Renditions: 100%|██████████| 8788/8788 [00:32<00:00, 273.14it/s]


# run the moments script from matlab
### navigate to the src directory in matlab, run the printed script, and hit enter once you return

In [ ]:
print(f"simple_region_props('{DIR_RENDITIONS}','{DIR_CSVS}','*.jpg')")

navigate to the src directory in matlab, run the printed script, and hit enter once you return 


simple_region_props('../data/test_dataset/output/renditions','../data/test_dataset/output/csvs','*.jpg')


In [ ]:
input("confirm that you have run the script")

# zip together the csvs

In [ ]:
CSV_REGION = join(DIR_CSVS, "fit_ellipse_moments.csv")

DF_REGION = pl.read_csv(CSV_REGION)


DF_MASTER = DF_ROTATIONS.join(
    DF_REGION,
    on='KID_IDFrameKey',
    how='inner'
).sort("Index")

DF_MASTER.write_csv(join(DIR_CSVS, "combined_dataset.csv"))
DF_MASTER

Index,KID_IDFrameKey,Object,Euler_X,Euler_Y,Euler_Z,image,centroid_x,centroid_y,axis_major_length,axis_minor_length,axis_ratio_minor_to_major,orientation_radians,orientation_degrees
i64,str,str,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,f64
0,"""0_CribNovel""","""CribNovel""",0,0,0,"""0_CribNovel.jpg""",412.067262,278.440639,105.729184,76.443295,0.72301,0.000376,0.021537
1,"""1_CribNovel""","""CribNovel""",-30,0,0,"""1_CribNovel.jpg""",411.701967,291.256074,104.709007,91.549189,0.87432,0.030473,1.745961
2,"""2_CribNovel""","""CribNovel""",-60,0,0,"""2_CribNovel.jpg""",411.305147,311.200608,105.735888,91.125251,0.86182,0.033152,1.899454
3,"""3_CribNovel""","""CribNovel""",-90,0,0,"""3_CribNovel.jpg""",412.442549,326.901344,115.348057,71.428241,0.619241,-0.004106,-0.235236
4,"""4_CribNovel""","""CribNovel""",-120,0,0,"""4_CribNovel.jpg""",411.299732,327.079881,109.97805,93.214542,0.847574,-0.005443,-0.311872
…,…,…,…,…,…,…,…,…,…,…,…,…,…
8783,"""8783_CameraNovel""","""CameraNovel""",180,30,30,"""8783_CameraNovel.jpg""",408.010113,316.830616,82.71998,31.62052,0.38226,0.466898,26.751306
8784,"""8784_CameraNovel""","""CameraNovel""",120,30,30,"""8784_CameraNovel.jpg""",407.372467,307.448669,79.11616,50.049199,0.632604,0.855754,49.031072
8785,"""8785_CameraNovel""","""CameraNovel""",90,30,30,"""8785_CameraNovel.jpg""",405.311555,302.1152,76.158048,61.997556,0.814064,0.835486,47.869817


# Now run the edge entropy script from matlab
### navigate to the src directory in matlab, run the printed script, and hit enter once you return

In [122]:
print(f"edge_entropy('{DIR_RENDITIONS}','{DIR_CSVS}','{DIR_2ND_ORDER}')")

edge_entropy('../data/test_dataset/output/renditions','../data/test_dataset/output/csvs','../data/test_dataset/output/csvs/second_order_entropy')


In [ ]:
input("confirm that you have run the script")